# Opening a new coffee shop in the City of Sydney, Australia

##  Introduction/Business Problem

Opening a coffee shops in the City of Sydney is a challenging and also potentially rewarding journey. Have you heard that Sydney is one of the cities with the best coffee? Personally I agree (Melbourne's and Aucklands are also good). It is a competitive business environment that customers are not only choosing the best cup of coffee, but also in a covenient location, and not to mention a good customer service as the basic of everything. This is a preliminary analysis for skilled coffee shop owners looking into expanding their business to the City of Syndey.

## Overview of how data will be collected and examined

First the locations, coordinates, and rating of all the coffee shops in the City of Sydney will be gathered via API from Foursquare (https://developer.foursquare.com/). Then the locations of these coffee shops will be split into 10 areas (village groups) as per the geojson file from the City of Sydney LGA (local government areas https://data.cityofsydney.nsw.gov.au/). With the total area and total employmnet data in each village known from the same data source, the number of coffee shop per employment ratio will be ploted in order to visually narrow down potential villages.

To leverage the data from Foursquare, location of train stations and bus stops will aslo be identified to estimate area with the best pedestrian traffic towards the potential villages. The ratings of exisitng coffee shops will be used as a guide to identify any final recommendation.

## Step by step plan on data collection and manipulation

1) Gather coffee shops data from Foursquare, using search end point, with limit and Query input (Applied Data Science Capstone, week 2).
2) Clean up data to a datafram that contains name of venue, unique ID, coordinates, and category (Applied Data Science Capstone, week 2)
3) Gather geojson data from API or a file from cityofsydney website, and display it using Folium (Data Visualization with Python, week 3).
4) Allocate coffee shops to each of the Village (Point in Polygon, online self study).
5) Count number of coffee shops in each Village. Gather number of employment and resident population in each village.
6) Examine data visually and decide what kind of regression can best describe the relationship by using employment and resident population to predict number of coffee shop in each Village.
7) Identify Villages (1-3) with coffee shops less than the predicted value as per the regression.
8) Ultilise premium call to get the average ratings of the coffee shops in these Village(s).
9) Find out which village has coffee shops with the least average ratings.
10) Additional findings on data.
11) The preliminary study will be concluded.